In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AL_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1517597200,AL,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,1.073003e+13
1,1798427201,AL,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,1.073014e+13
2,2886767410,AL,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,1.117030e+13
3,4982277110,AL,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,1.073003e+13
4,5001327110,AL,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,1.073013e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1517597200,AL,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,1.073003e+13
1,1798427201,AL,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,1.073014e+13
2,2886767410,AL,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,1.117030e+13
3,4982277110,AL,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,1.073003e+13
4,5001327110,AL,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,1.073013e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
212,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1097,6283588301,AL,"10601 Highway 43, Creola, AL",30.912869,-88.036347,NaN
1277,8999377001,AL,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,9686247001,AL,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,1300428408,AL,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,9072157102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,6703258010,AL,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,8161698300,AL,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,7149097206,AL,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('01')] #AL
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/33 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

33


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10722/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
212,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,[]
572,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,[]
1097,6283588301,AL,"10601 Highway 43, Creola, AL",30.912869,-88.036347,[010970057022055]
1277,8999377001,AL,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,[]
1507,9686247001,AL,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,[]
1831,1300428408,AL,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,[]
2586,9072157102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,[]
3141,6703258010,AL,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,[]
3144,8161698300,AL,"509, MADISON, AL",40.759209,-73.974410,[]
3272,7149097206,AL,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_10722/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
212,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1097,6283588301,AL,"10601 Highway 43, Creola, AL",30.912869,-88.036347,010970057022055
1277,8999377001,AL,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,9686247001,AL,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,1300428408,AL,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,9072157102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,6703258010,AL,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,8161698300,AL,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,7149097206,AL,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
212,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1277,8999377001,AL,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,9686247001,AL,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,1300428408,AL,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,9072157102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,6703258010,AL,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,8161698300,AL,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,7149097206,AL,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN
4870,6549927102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
212,6205747005,NaN
572,5260357102,NaN
1097,6283588301,010970057022055
1277,8999377001,NaN
1507,9686247001,NaN
1831,1300428408,NaN
2586,9072157102,NaN
3141,6703258010,NaN
3144,8161698300,NaN
3272,7149097206,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/AL_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,2662,1517597200,AL,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,1.073003e+13
1,2663,1798427201,AL,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,1.073014e+13
2,2664,2886767410,AL,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,1.117030e+13
3,2665,4982277110,AL,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,1.073003e+13
4,2666,5001327110,AL,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,1.073013e+13


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,2662,1517597200,AL,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,1.073003e+13,NaN
1,2663,1798427201,AL,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,1.073014e+13,NaN
2,2664,2886767410,AL,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,1.117030e+13,NaN
3,2665,4982277110,AL,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,1.073003e+13,NaN
4,2666,5001327110,AL,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,1.073013e+13,NaN
...,...,...,...,...,...,...,...,...
10907,13569,9206768503,AL,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696,1.085781e+13,NaN
10908,13570,9298908504,AL,"685 Fox Trl, Opelika, AL",32.650615,-85.358977,1.081042e+13,NaN
10909,13571,9323947403,AL,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368,1.117030e+13,NaN
10910,13572,9406888509,AL,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929,1.097007e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,2662,1517597200,AL,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,1.073003e+13,NaN,10730027012041.0
1,2663,1798427201,AL,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,1.073014e+13,NaN,10730144043001.0
2,2664,2886767410,AL,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,1.117030e+13,NaN,11170302122003.0
3,2665,4982277110,AL,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,1.073003e+13,NaN,10730027025027.0
4,2666,5001327110,AL,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,1.073013e+13,NaN,10730129161002.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1517597200,AL,"204 20TH ST N, BIRMINGHAM, AL",33.515571,-86.806634,10730027012041.0
1,1798427201,AL,"2090 Columbiana Road, VESTAVIA HILLS, AL",33.435789,-86.811428,10730144043001.0
2,2886767410,AL,"3700 Cahaba Beach Road, BIRMINGHAM, AL",33.431916,-86.689670,11170302122003.0
3,4982277110,AL,"1901 6TH AVE N Ste 2400, BIRMINGHAM, AL",33.518619,-86.809388,10730027025027.0
4,5001327110,AL,"VESTAVIA PKWY, VESTAVIA, AL",33.418736,-86.797244,10730129161002.0
...,...,...,...,...,...,...
10907,9206768503,AL,"1575 Old Fort Rd E, Fort Deposit, AL",31.975720,-86.603696,10857812001055.0
10908,9298908504,AL,"685 Fox Trl, Opelika, AL",32.650615,-85.358977,10810416003005.0
10909,9323947403,AL,"221 Wagon Trail, Alabaster, AL",33.256737,-86.828368,11170303201008.0
10910,9406888509,AL,"5685B I 10 Industrial Pkwy N, Theodore, AL",30.568623,-88.175929,10970071012016.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
212,6205747005,AL,"1980 hwy 315, SAGINAW, AL",34.212860,-89.696248,NaN
572,5260357102,AL,"5015 Business Park BLVD, NEW MARKET, AL",61.174949,-149.888309,NaN
1277,8999377001,AL,"508 GREENE STREET, GREENSBORO, AL",36.078119,-79.790771,NaN
1507,9686247001,AL,"11180 Highway 82, MC SHAN, AL",33.352678,-93.396758,NaN
1831,1300428408,AL,"10350 Highway 80 E, Montgomery, AL",32.715967,-96.120698,NaN
2586,9072157102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN
3141,6703258010,AL,"4100 GEORGE J. BEAN PKWY SPACE 1-2-2687, MOBIL...",27.977152,-82.531138,NaN
3144,8161698300,AL,"509, MADISON, AL",40.759209,-73.974410,NaN
3272,7149097206,AL,"11070 Hwy 80 East, MOUNT MEIGS, AL",32.358810,-89.549259,NaN
4870,6549927102,AL,"450 STATE LINE RD, WILMER, AL",30.675558,-88.412497,NaN


Convert to File

In [19]:
fips_AL = fips_merge_drop

In [20]:
fips_AL.to_csv("../../../data/state_data/geo/geo_fips/23/AL_fips_scraped.csv") 